This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/computer-vision).

## Step 01: Detecting Objects Using mobilenet

The following tutorial demonstrates how to use a trained mobilenet model deployed in Wallaroo to detect objects.  This process will use the following steps:

1. Create a Wallaroo workspace and pipeline.
1. Upload a trained mobilenet ML model and add it as a pipeline step.
1. Deploy the pipeline.
1. Perform an inference on a sample image.
1. Draw the detected objects, their bounding boxes, their classifications, and the confidence of the classifications on the provided image.
1. Review our results.

## Steps

### Import Libraries

The first step will be to import our libraries.  Please check with **Step 00: Introduction and Setup** and verify that the necessary libraries and applications are added to your environment.

In [1]:
import torch
import pickle
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

import numpy as np
import json
import requests
import time
import pandas as pd
from CVDemoUtils import CVDemo

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

# used for unique connection names

import string
import random
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))
suffix= 'jch'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local service

wl = wallaroo.Client()


wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Variables

The following variables and methods are used later to create or connect to an existing workspace, pipeline, and model.

In [3]:
workspace_name = f'mobilenetworkspacetest{suffix}'
pipeline_name = f'mobilenetpipeline{suffix}'
model_name = f'mobilenet{suffix}'
model_file_name = 'models/mobilenet.pt.onnx'

In [4]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

### Create Workspace

The workspace will be created or connected to, and set as the default workspace for this session.  Once that is done, then all models and pipelines will be set in that workspace.

In [5]:
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)
wl.get_current_workspace()

{'name': 'mobilenetworkspacetestjch', 'id': 7, 'archived': False, 'created_by': '0a36fba2-ad42-441b-9a8c-bac8c68d13fa', 'created_at': '2023-07-28T16:43:15.483554+00:00', 'models': [], 'pipelines': []}

### Create Pipeline and Upload Model

We will now create or connect to an existing pipeline as named in the variables above.

In [6]:
pipeline = get_pipeline(pipeline_name)
mobilenet_model = wl.upload_model(model_name, model_file_name, framework=Framework.ONNX).configure(batch_config="single")

### Deploy Pipeline

With the model uploaded, we can add it is as a step in the pipeline, then deploy it.  Once deployed, resources from the Wallaroo instance will be reserved and the pipeline will be ready to use the model to perform inference requests. 

In [7]:
pipeline.add_model_step(mobilenet_model)

pipeline.deploy()

name,mobilenetpipelinejch
created,2023-07-28 16:44:07.799676+00:00
last_updated,2023-07-28 16:44:07.799676+00:00
deployed,True
tags,
versions,e723a107-2827-4399-8238-8e048e938ae8
steps,mobilenetjch


### Prepare input image

Next we will load a sample image and resize it to the width and height required for the object detector.  Once complete, it the image will be converted to a numpy ndim array and added to a dictionary.

In [8]:

# The size the image will be resized to
width = 640
height = 480

# Only objects that have a confidence > confidence_target will be displayed on the image
cvDemo = CVDemo()

imagePath = 'data/images/input/example/dairy_bottles.png'

# The image width and height needs to be set to what the model was trained for.  In this case 640x480.
tensor, resizedImage = cvDemo.loadImageAndResize(imagePath, width, height)

# get npArray from the tensorFloat
npArray = tensor.cpu().numpy()

#creates a dictionary with the wallaroo "tensor" key and the numpy ndim array representing image as the value.

dictData = {"tensor":[npArray]}
dataframedata = pd.DataFrame(dictData)


### Run Inference

With that done, we can have the model detect the objects on the image by running an inference through the pipeline, and storing the results for the next step.

In [9]:
startTime = time.time()
# pass the dataframe in 
infResults = pipeline.infer(dataframedata, dataset=["*", "metadata.elapsed"])
endTime = time.time()

In [10]:
infResults

,time,in.tensor,out.boxes,out.classes,out.confidences,check_failures,metadata.elapsed
0,2023-07-28 16:44:27.879,"[0.9372549057, 0.9529411793, 0.9490196109, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9490196109, 0.9490196109, 0.9529411793, 0.9529411793, 0.9490196109, 0.9607843161, 0.9686274529, 0.9647058845, 0.9686274529, 0.9647058845, 0.9568627477, 0.9607843161, 0.9647058845, 0.9647058845, 0.9607843161, 0.9647058845, 0.9725490212, 0.9568627477, 0.9607843161, 0.9176470637, 0.9568627477, 0.9176470637, 0.8784313798, 0.8941176534, 0.8431372643, 0.8784313798, 0.8627451062, 0.850980401, 0.9254902005, 0.8470588326, 0.9686274529, 0.8941176534, 0.8196078539, 0.850980401, 0.9294117689, 0.8666666746, 0.8784313798, 0.8666666746, 0.9647058845, 0.9764705896, 0.980392158, 0.9764705896, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.980392158, 0.8941176534, 0.4823529422, 0.4627451003, 0.4313725531, 0.270588249, 0.2588235438, 0.2941176593, 0.3450980484, 0.3686274588, 0.4117647111, 0.4549019635, 0.4862745106, 0.5254902244, 0.5607843399, 0.6039215922, 0.6470588446, 0.6862745285, 0.721568644, 0.7450980544, 0.7490196228, 0.7882353067, 0.8666666746, 0.980392158, 0.9882352948, 0.9686274529, 0.9647058845, 0.9686274529, 0.9725490212, 0.9647058845, 0.9607843161, 0.9607843161, 0.9607843161, 0.9607843161, ...]","[0.0, 210.2901, 85.26463, 479.07495, 72.03781, 197.3227, 151.44223, 468.43225, 211.28015, 184.72838, 277.2192, 420.42746, 143.23904, 203.83005, 216.85547, 448.8881, 13.095016, 41.91339, 640.0, 480.0, 106.51464, 206.14499, 159.54643, 463.96756, 278.0637, 1.5217237, 321.45782, 93.563484, 462.31833, 104.16202, 510.5396, 224.75314, 310.4559, 1.3959818, 352.8513, 94.123825, 528.0485, 268.4225, 636.26715, 475.7666, 220.06293, 0.51385117, 258.31833, 90.18019, 552.87115, 96.30235, 600.7256, 233.53384, 349.24072, 0.27034378, 404.17325, 98.68022, 450.89346, 264.2356, 619.6033, 472.65173, 261.51385, 193.4335, 307.17914, 408.75247, 509.22018, 101.16539, 544.1857, 235.7374, 592.4824, 100.38687, 633.77985, 239.13432, 475.54208, 297.6141, 551.0544, 468.01547, 368.81982, 163.61407, 423.90942, 362.7888, 120.669, 0.0, 175.9362, 81.774574, 72.48429, 0.0, 143.50789, 85.4698, 271.12686, 200.89185, 305.626, 274.59537, 161.80728, 0.0, 213.08308, 85.42828, 162.13324, 0.0, 214.60814, 83.81444, 310.89108, 190.95468, 367.32925, 397.28137, ...]","[44, 44, 44, 44, 82, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 84, 84, 44, 84, 44, 44, 44, 61, 44, 86, 44, 44]","[0.98649, 0.90115356, 0.6077846, 0.5922323, 0.53729033, 0.4513168, 0.43728516, 0.43094054, 0.4084834, 0.39185277, 0.35759133, 0.3181266, 0.26451287, 0.23062895, 0.20482065, 0.174621, 0.17313862, 0.15999581, 0.14913696, 0.1366402, 0.13322707, 0.12218643, 0.121301256, 0.11956108, 0.11527827, 0.09616333, 0.08654833, 0.078406945, 0.07234089, 0.062820904, 0.052787986]",0,"[173453264, 71644735]"


### Draw the Inference Results

With our inference results, we can take them and use the Wallaroo CVDemo class and draw them onto the original image.  The bounding boxes and the confidence value will only be drawn on images where the model returned a 90% confidence rate in the object's identity.

In [12]:
elapsed = 1.0
#elapsed = infResults['elapsed']
results = {
    'model_name' : model_name,
    'pipeline_name' : pipeline_name,
    'width': width,
    'height': height,
    'image' : resizedImage,
    'inf-results' : infResults,
    'confidence-target' : 0.50,
    'inference-time': (endTime-startTime),
    'onnx-time' : int(elapsed) / 1e+9,                
    'color':(255,0,0)
}

image = cvDemo.drawDetectedObjectsFromInference(results)

KeyError: 'outputs'

### Extract the Inference Information

To show what is going on in the background, we'll extract the inference results create a dataframe with columns representing the classification, confidence, and bounding boxes of the objects identified.

In [17]:
idx = 0 
for idx in range(0,len(classes)):
    df['classification'][idx] = cvDemo.CLASSES[classes[idx]] # Classes contains the 80 different COCO classificaitons
    df['confidence'][idx] = confidences[idx]
df

,classification,confidence,x,y,width,height
0,bottle,98.65%,0,210,85,479
1,bottle,90.12%,72,197,151,468
2,bottle,60.78%,211,184,277,420
3,bottle,59.22%,143,203,216,448
4,refrigerator,53.73%,13,41,640,480
5,bottle,45.13%,106,206,159,463
6,bottle,43.73%,278,1,321,93
7,bottle,43.09%,462,104,510,224
8,bottle,40.85%,310,1,352,94
9,bottle,39.19%,528,268,636,475


### Undeploy the Pipeline

With the inference complete, we can undeploy the pipeline and return the resources back to the Wallaroo instance.

In [18]:
pipeline.undeploy()

name,mobilenetpipelinehsec
created,2023-07-14 15:14:45.891178+00:00
last_updated,2023-07-14 15:15:07.488456+00:00
deployed,False
tags,
versions,"13e17927-aef3-410c-b695-6bd713248f93, ae7cd8ab-7ad0-48ce-ba01-8e8e12595f45"
steps,mobilenethsec
